In [61]:
## Credit: Coursera

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Flatten, Embedding
from keras.models import Sequential
import io

In [10]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

train_sentences = []
train_labels = []
test_sentences = []
test_labels = []

for s,l in train_data:
    train_sentences.append(str(s.numpy()))
    train_labels.append(l.numpy())
    
for s,l in test_data:
    test_sentences.append(str(s.numpy()))
    test_labels.append(l.numpy())
    
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [35]:
vocab_size = 10000
out_of_vocab = '<OOV>'
truncate_mode = 'post'
max_len = 130
embedding_dim = 16

tokenizer = Tokenizer(num_words=vocab_size, oov_token=out_of_vocab)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
sequences = pad_sequences(sequences, maxlen=max_len)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_sequences = pad_sequences(test_sequences, maxlen=max_len)

In [41]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 130, 16)           160000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 2080)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 12486     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 7         
Total params: 172,493
Trainable params: 172,493
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(sequences, train_labels, epochs=10, validation_data=(test_sequences, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 156us/step - loss: 5.9716e-05 - accuracy: 1.0000 - val_loss: 0.7720 - val_accuracy: 0.8483
Epoch 2/10
25000/25000 [==============================] - 4s 156us/step - loss: 3.7601e-05 - accuracy: 1.0000 - val_loss: 0.8025 - val_accuracy: 0.8482
Epoch 3/10
25000/25000 [==============================] - 4s 157us/step - loss: 2.3879e-05 - accuracy: 1.0000 - val_loss: 0.8303 - val_accuracy: 0.8479
Epoch 4/10
25000/25000 [==============================] - 4s 156us/step - loss: 1.5301e-05 - accuracy: 1.0000 - val_loss: 0.8601 - val_accuracy: 0.8480
Epoch 5/10
25000/25000 [==============================] - 4s 154us/step - loss: 9.8048e-06 - accuracy: 1.0000 - val_loss: 0.8880 - val_accuracy: 0.8477
Epoch 6/10
25000/25000 [==============================] - 4s 159us/step - loss: 6.2513e-06 - accuracy: 1.0000 - val_loss: 0.9186 - val_accuracy: 0.8479
Epoch 7/10
25000/25000 [==============

In [64]:
e = model.layers[0]
weights = e.get_weights()[0]

out_v = io.open('vecs.tsv', 'w')
out_m = io.open('meta.tsv', 'w')

reverse_word_index = dict([(value, word) for (word, value) in word_index.items()])

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_v.write(word + '\n')
    out_m.write('\t'.join([str(x) for x in embeddings]) + '\n')

out_m.close()
out_v.close()